In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Feature extraction

In [2]:
import glob
import os
import numpy as np
from pathlib import Path
from PIL import Image, ImageFile
from omegaconf import OmegaConf

from config.init import create_baseconfig_from_checkpoint
from model.lgffem import LGFFEM

from utils.revisitop.dataset import configdataset
from utils.revisitop.evaluate import compute_map

from tqdm import tqdm

import faiss

In [3]:
import torch
from torchvision.transforms import v2
from torch.utils.data import Dataset

from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def pil_loader(path):
    # to avoid crashing for truncated (corrupted images)
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    # open path as file to avoid ResourceWarning 
    # (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

In [5]:
def revised_metrics_from_checkpoint(l_checkpoint: list[str], test_dataset = 'roxford5k', retrieval_option='default'):

    for path_checkpoint in l_checkpoint:
        
        # Create model from checkpoint
        print(path_checkpoint)
        path_checkpoint = os.path.join(path_checkpoint)
        checkpoint = torch.load(path_checkpoint)
        
        base_config = create_baseconfig_from_checkpoint(checkpoint)

        embedder = LGFFEM(base_config).eval().to(device)
        match_n = embedder.neck.load_state_dict(checkpoint['model_neck_state_dict'], strict = False)
        print('[++] Loaded neck weights.', match_n)
        match_h = embedder.head.load_state_dict(checkpoint['model_head_state_dict'], strict = False)
        print('[++] Loaded head weights.', match_h)
        
        img_transforms = v2.Compose([
                            v2.ToImage(),
                            v2.Resize(size=(224, 224)),
                            v2.ToDtype(torch.float32, scale=True),
                            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                          ])
        
        # Extraction features dataset
        data_root = os.path.join('/thesis/classical/revisitop')
        
        print('>> {}: Processing test dataset...'.format(test_dataset)) 
        # config file for the dataset
        # separates query image list from database image list, if revisited protocol used
        cfg = configdataset(test_dataset, os.path.join(data_root, 'datasets'))
        
        # query images
        qim_embs_l = []
        for i in tqdm(np.arange(cfg['nq'])):
            qim = pil_loader(cfg['qim_fname'](cfg, i)).crop(cfg['gnd'][i]['bbx'])
            qim = img_transforms(qim).unsqueeze(0).to(device)
            qim_embs_l.append(embedder(qim).detach().cpu().numpy())

        Q = np.swapaxes(np.stack(qim_embs_l, axis=0).squeeze(1),0,1)
        
        im_embs_l = []
        for i in tqdm(np.arange(cfg['n'])):
            im = pil_loader(cfg['im_fname'](cfg, i))
            im = img_transforms(im).unsqueeze(0).to(device)
            im_embs_l.append(embedder(im).detach().cpu().numpy())

        X = np.swapaxes(np.stack(im_embs_l, axis=0).squeeze(1),0,1)
        
        if retrieval_option=='default':
            sim = np.dot(X.T, Q)
            ranks_f = np.argsort(-sim, axis=0)

        elif retrieval_option=='faiss':
            dim, size = X.shape
            index = faiss.IndexFlatL2(dim) ##euclidean
    #         index = faiss.IndexFlatIP(dim) ##cosine
            index.train(np.swapaxes(X,0,1))
            index.add(np.swapaxes(X,0,1))
            sims, ranks_f = index.search(np.swapaxes(Q,0,1), size)
            ranks_f = np.swapaxes(ranks_f,0,1)
            
        else:
            raise Exception("The retrieval_option selected doesn't exist.")
        
        # revisited evaluation
        gnd = cfg['gnd']

        # evaluate ranks
        ks = [1, 5, 10]
        
        # search for easy
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['easy']])
            g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
            gnd_t.append(g)
        mapE, apsE, mprE, prsE = compute_map(ranks_f, gnd_t, ks)
        
        # search for easy & hard
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk']])
            gnd_t.append(g)
        mapM, apsM, mprM, prsM = compute_map(ranks_f, gnd_t, ks)
        
        # search for hard
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
            gnd_t.append(g)
        mapH, apsH, mprH, prsH = compute_map(ranks_f, gnd_t, ks)
        
        print(f'>> {Path(path_checkpoint).stem}')
        print('>> {}: mAP E: {}, M: {}, H: {}'.format(test_dataset, np.around(mapE*100, decimals=2), np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
        print('>> {}: mP@k{} E: {}, M: {}, H: {}'.format(test_dataset, np.array(ks), np.around(mprE*100, decimals=2), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))
        print('\n\n')

In [6]:
kimia_m_checks = [
                '/thesis/checkpoint/BASE/BASE-A-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch200.pth',
                '/thesis/checkpoint/BASE/BASE-B-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch198.pth',
                '/thesis/checkpoint/BASE/BASE-C-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch199.pth'
               ]

In [7]:
revised_metrics_from_checkpoint(kimia_m_checks)

/thesis/checkpoint/BASE/BASE-A-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch200.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> roxford5k: Processing test dataset...


100%|██████████| 4993/4993 [02:22<00:00, 34.95it/s]


>> BASE-A-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch200
>> roxford5k: mAP E: 10.62, M: 9.7, H: 2.97
>> roxford5k: mP@k[ 1  5 10] E: [22.06 21.76 17.94], M: [25.71 23.14 19.43], H: [8.57 4.86 3.57]



/thesis/checkpoint/BASE/BASE-B-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch198.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> roxford5k: Processing test dataset...


100%|██████████| 4993/4993 [02:26<00:00, 34.11it/s]


>> BASE-B-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch198
>> roxford5k: mAP E: 7.86, M: 7.82, H: 2.3
>> roxford5k: mP@k[ 1  5 10] E: [14.71 15.   13.68], M: [18.57 16.57 15.29], H: [5.71 2.29 2.43]



/thesis/checkpoint/BASE/BASE-C-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch199.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> roxford5k: Processing test dataset...


100%|██████████| 4993/4993 [02:26<00:00, 34.17it/s]


>> BASE-C-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch199
>> roxford5k: mAP E: 10.74, M: 9.95, H: 2.04
>> roxford5k: mP@k[ 1  5 10] E: [20.59 17.65 16.47], M: [21.43 18.86 17.43], H: [2.86 3.14 3.14]





In [8]:
revised_metrics_from_checkpoint(kimia_m_checks, test_dataset='rparis6k')

/thesis/checkpoint/BASE/BASE-A-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch200.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> rparis6k: Processing test dataset...


100%|██████████| 6322/6322 [02:59<00:00, 35.19it/s]


>> BASE-A-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch200
>> rparis6k: mAP E: 45.27, M: 36.72, H: 14.96
>> rparis6k: mP@k[ 1  5 10] E: [88.57 77.71 72.  ], M: [88.57 82.57 75.71], H: [35.71 34.29 30.71]



/thesis/checkpoint/BASE/BASE-B-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch198.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> rparis6k: Processing test dataset...


100%|██████████| 6322/6322 [03:05<00:00, 34.16it/s]


>> BASE-B-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch198
>> rparis6k: mAP E: 28.69, M: 24.86, H: 10.16
>> rparis6k: mP@k[ 1  5 10] E: [80.   71.14 64.71], M: [81.43 75.71 69.43], H: [42.86 34.29 30.  ]



/thesis/checkpoint/BASE/BASE-C-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch199.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> rparis6k: Processing test dataset...


100%|██████████| 6322/6322 [03:01<00:00, 34.79it/s]


>> BASE-C-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch199
>> rparis6k: mAP E: 31.31, M: 24.61, H: 7.98
>> rparis6k: mP@k[ 1  5 10] E: [80.   72.57 66.57], M: [81.43 74.57 69.57], H: [27.14 24.   21.71]





In [9]:
final_models = ['/thesis/checkpoint/BASE/BASE-EMB-11_convnextv2-02_neck_512_3-00_head_A-epoch10--in1k-224.pth',
 '/thesis/checkpoint/BASE/BASE-B-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch198.pth',
 '/thesis/checkpoint/BASE/BASE-EMB-KIMIA-11_convnextv2-02_neck_512_3-00_head_A-epoch499.pth'
 ]

In [10]:
revised_metrics_from_checkpoint(final_models)

/thesis/checkpoint/BASE/BASE-EMB-11_convnextv2-02_neck_512_3-00_head_A-epoch10--in1k-224.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> roxford5k: Processing test dataset...


100%|██████████| 4993/4993 [02:20<00:00, 35.59it/s]


>> BASE-EMB-11_convnextv2-02_neck_512_3-00_head_A-epoch10--in1k-224
>> roxford5k: mAP E: 19.78, M: 14.6, H: 3.56
>> roxford5k: mP@k[ 1  5 10] E: [29.41 24.85 23.82], M: [28.57 25.71 24.14], H: [7.14 7.14 6.71]



/thesis/checkpoint/BASE/BASE-B-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch198.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> roxford5k: Processing test dataset...


100%|██████████| 4993/4993 [02:23<00:00, 34.76it/s]


>> BASE-B-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch198
>> roxford5k: mAP E: 7.86, M: 7.82, H: 2.3
>> roxford5k: mP@k[ 1  5 10] E: [14.71 15.   13.68], M: [18.57 16.57 15.29], H: [5.71 2.29 2.43]



/thesis/checkpoint/BASE/BASE-EMB-KIMIA-11_convnextv2-02_neck_512_3-00_head_A-epoch499.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> roxford5k: Processing test dataset...


100%|██████████| 4993/4993 [02:37<00:00, 31.65it/s]


>> BASE-EMB-KIMIA-11_convnextv2-02_neck_512_3-00_head_A-epoch499
>> roxford5k: mAP E: 2.7, M: 3.39, H: 1.18
>> roxford5k: mP@k[ 1  5 10] E: [5.88 3.82 4.26], M: [5.71 4.57 5.  ], H: [0.   1.14 1.  ]





In [11]:
revised_metrics_from_checkpoint(final_models, test_dataset='rparis6k')

/thesis/checkpoint/BASE/BASE-EMB-11_convnextv2-02_neck_512_3-00_head_A-epoch10--in1k-224.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> rparis6k: Processing test dataset...


100%|██████████| 6322/6322 [03:19<00:00, 31.75it/s]


>> BASE-EMB-11_convnextv2-02_neck_512_3-00_head_A-epoch10--in1k-224
>> rparis6k: mAP E: 54.64, M: 43.68, H: 20.6
>> rparis6k: mP@k[ 1  5 10] E: [91.43 85.43 82.14], M: [91.43 86.86 84.43], H: [58.57 51.43 46.71]



/thesis/checkpoint/BASE/BASE-B-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch198.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> rparis6k: Processing test dataset...


100%|██████████| 6322/6322 [03:07<00:00, 33.73it/s]


>> BASE-B-EMB-PAN-11_convnextv2-02_neck_512_3-00_head_A-epoch198
>> rparis6k: mAP E: 28.69, M: 24.86, H: 10.16
>> rparis6k: mP@k[ 1  5 10] E: [80.   71.14 64.71], M: [81.43 75.71 69.43], H: [42.86 34.29 30.  ]



/thesis/checkpoint/BASE/BASE-EMB-KIMIA-11_convnextv2-02_neck_512_3-00_head_A-epoch499.pth
[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>
>> rparis6k: Processing test dataset...


100%|██████████| 6322/6322 [03:09<00:00, 33.40it/s]


>> BASE-EMB-KIMIA-11_convnextv2-02_neck_512_3-00_head_A-epoch499
>> rparis6k: mAP E: 7.63, M: 8.67, H: 4.48
>> rparis6k: mP@k[ 1  5 10] E: [32.86 23.14 19.07], M: [34.29 27.71 24.43], H: [12.86  7.71  7.57]



